In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Import data

In [3]:
data = pd.read_csv('MarketPricePrediction.csv')

In [4]:
data.shape

(10227, 10)

## Preprocessing

In [5]:
data.isnull().sum()

market      0
month       0
year        0
quantity    0
priceMin    0
priceMax    0
priceMod    0
state       0
city        0
date        0
dtype: int64

In [6]:
data = data.drop('date', axis = 1)
data.head()

,market,month,year,quantity,priceMin,priceMax,priceMod,state,city
0,ABOHAR(PB),January,2005,2350,404,493,446,PB,ABOHAR
1,ABOHAR(PB),January,2006,900,487,638,563,PB,ABOHAR
2,ABOHAR(PB),January,2010,790,1283,1592,1460,PB,ABOHAR
3,ABOHAR(PB),January,2011,245,3067,3750,3433,PB,ABOHAR
4,ABOHAR(PB),January,2012,1035,523,686,605,PB,ABOHAR


In [7]:
data['city'].unique().shape

(117,)

In [8]:
encoder = LabelEncoder()

In [9]:
data['city'] = encoder.fit_transform(data['city'])
data['month'] = encoder.fit_transform(data['month'])
data['market'] = encoder.fit_transform(data['market'])
data['state'] = encoder.fit_transform(data['state'])

In [10]:
data.head()

,market,month,year,quantity,priceMin,priceMax,priceMod,state,city
0,0,4,2005,2350,404,493,446,16,0
1,0,4,2006,900,487,638,563,16,0
2,0,4,2010,790,1283,1592,1460,16,0
3,0,4,2011,245,3067,3750,3433,16,0
4,0,4,2012,1035,523,686,605,16,0


### sort values

In [11]:
df = data.sort_values(by = ["year", "month"])

In [12]:
# Créez les nouvelles colonnes
df['prev_quantity'] = df.groupby(['market', 'year'])['quantity'].shift(1)
df['prev_priceMin'] = df.groupby(['market', 'year'])['priceMin'].shift(1)
df['prev_priceMax'] = df.groupby(['market', 'year'])['priceMax'].shift(1)
df['prev_priceMod'] = df.groupby(['market', 'year'])['priceMod'].shift(1)

In [13]:
df['prev_quantity'] = df.groupby(['month', 'market'])['prev_quantity'].fillna(method='ffill')
df['prev_priceMin'] = df.groupby(['month', 'market'])['prev_priceMin'].fillna(method='ffill')
df['prev_priceMax'] = df.groupby(['month', 'market'])['prev_priceMax'].fillna(method='ffill')
df['prev_priceMod'] = df.groupby(['month', 'market'])['prev_priceMod'].fillna(method='ffill')

In [14]:
df = df.dropna()

In [15]:
order = ['market', 'month', 'year', 'prev_quantity', 'prev_priceMin', 'prev_priceMax',
       'prev_priceMod', 'state', 'city', 'quantity', 'priceMin',
       'priceMax', 'priceMod']

In [16]:
df = df[order]

In [17]:
df.head()

,market,month,year,prev_quantity,prev_priceMin,prev_priceMax,prev_priceMod,state,city,quantity,priceMin,priceMax,priceMod
5541,68,1,1996,192592.0,136.0,279.0,254.0,14,65,173892,164,388,340
5621,68,2,1996,173892.0,164.0,388.0,340.0,14,65,240615,138,451,385
5420,68,3,1996,240615.0,138.0,451.0,385.0,14,65,196164,133,229,186
5399,68,4,1996,196164.0,133.0,229.0,186.0,14,65,225063,160,257,226
5521,68,5,1996,225063.0,160.0,257.0,226.0,14,65,156282,229,406,368


## Train and test data

In [18]:
X = df.iloc[:,:9]
y = df.iloc[:,9:]

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

# Model Extra trees

In [20]:
model = MultiOutputRegressor(ExtraTreesRegressor())

In [21]:
model.fit(x_train, y_train)

MultiOutputRegressor(estimator=ExtraTreesRegressor())

In [22]:
model.score(x_train, y_train)

0.9999999987929415

In [23]:
predictions = model.predict(x_test)

In [24]:
predictions

array([[3.0101902e+05, 8.0367000e+02, 2.3105000e+03, 1.7967700e+03],
       [6.7697400e+03, 1.7271700e+03, 1.7837600e+03, 1.6616400e+03],
       [2.5060910e+04, 2.6740000e+02, 6.4443000e+02, 4.8733000e+02],
       ...,
       [7.2746200e+03, 2.5276000e+02, 4.4932000e+02, 3.6702000e+02],
       [2.0344771e+05, 1.8337000e+02, 1.9929300e+03, 1.0884400e+03],
       [1.2737780e+04, 4.0651000e+02, 1.3396300e+03, 9.3971000e+02]])

## Model evaluation

In [25]:
np.sqrt(mean_squared_error(predictions, y_test))

32427.638134911995

In [26]:
mean_absolute_error(predictions, y_test)

8442.821452865559